# Clase Completa de Apache Airflow

## Introducción

Apache Airflow es una plataforma de código abierto para programar, orquestar y monitorear flujos de trabajo (workflows) de forma programática.

### ¿Qué aprenderemos?
1. Conceptos básicos de Airflow
2. Instalación y configuración
3. DAGs (Directed Acyclic Graphs)
4. Operadores y tareas
5. Dependencias y ejecución
6. Ejemplos prácticos

## 1. Instalación de Apache Airflow

Primero instalamos Airflow. Es recomendable usar un entorno virtual.

In [ ]:
# ============================================
# INSTALACIÓN DE APACHE AIRFLOW
# ============================================
# NOTA: Ejecutar esto solo una vez para instalar las dependencias

# Importamos sys para obtener la ruta del intérprete de Python actual
import sys

# Instalamos Apache Airflow versión 2.7.3
# !{sys.executable} ejecuta pip con el Python actual del notebook
!{sys.executable} -m pip install apache-airflow==2.7.3

# Instalamos el provider de HTTP para hacer requests a APIs
!{sys.executable} -m pip install apache-airflow-providers-http

## 2. Conceptos Fundamentales

### DAG (Directed Acyclic Graph)
- Es un grafo dirigido sin ciclos que representa un flujo de trabajo
- Contiene tareas y sus dependencias

### Tarea (Task)
- Unidad básica de trabajo en Airflow
- Se define usando operadores

### Operador (Operator)
- Template para una tarea
- Define QUÉ se ejecutará

### Scheduler
- Componente que programa la ejecución de los DAGs

## 3. Configuración Inicial

In [ ]:
# ============================================
# IMPORTAR LIBRERÍAS NECESARIAS
# ============================================

# datetime y timedelta: Para manejar fechas y duraciones de tiempo
from datetime import datetime, timedelta

# DAG: Clase principal para definir un flujo de trabajo en Airflow
from airflow import DAG

# PythonOperator: Ejecuta funciones Python como tareas
from airflow.operators.python import PythonOperator

# BashOperator: Ejecuta comandos de línea de comandos (bash/shell)
from airflow.operators.bash import BashOperator

# DummyOperator: Tarea vacía que no hace nada, útil para organizar el flujo
from airflow.operators.dummy import DummyOperator

print("Librerías importadas correctamente")

## 4. Argumentos por Defecto

Los argumentos por defecto se aplican a todas las tareas del DAG si no se especifica lo contrario.

In [ ]:
# ============================================
# ARGUMENTOS POR DEFECTO DEL DAG
# ============================================
# Estos parámetros se aplican a TODAS las tareas del DAG automáticamente

default_args = {
    # 'owner': Propietario del DAG (nombre del equipo o persona)
    'owner': 'airflow',
    
    # 'depends_on_past': Si True, la tarea solo se ejecuta si la anterior fue exitosa
    'depends_on_past': False,
    
    # 'start_date': Fecha desde la cual el DAG está activo
    'start_date': datetime(2024, 1, 1),
    
    # 'email': Lista de correos para notificaciones
    'email': ['admin@ejemplo.com'],
    
    # 'email_on_failure': Enviar email cuando una tarea falla
    'email_on_failure': False,
    
    # 'email_on_retry': Enviar email cuando una tarea se reintenta
    'email_on_retry': False,
    
    # 'retries': Número de reintentos si una tarea falla
    'retries': 1,
    
    # 'retry_delay': Tiempo de espera entre reintentos
    'retry_delay': timedelta(minutes=5)
}

# Mostrar la configuración
print("Argumentos por defecto configurados:")
for key, value in default_args.items():
    print(f"  {key}: {value}")

## 5. Crear tu Primer DAG

Vamos a crear un DAG simple que ejecuta varias tareas.

In [ ]:
# ============================================
# CREAR EL PRIMER DAG
# ============================================

# Instanciamos un objeto DAG con sus parámetros
dag = DAG(
    # 'dag_id': Identificador único del DAG (debe ser único en Airflow)
    'mi_primer_dag',
    
    # Heredamos los argumentos por defecto definidos anteriormente
    default_args=default_args,
    
    # Descripción que aparecerá en la UI de Airflow
    description='Un DAG simple de ejemplo',
    
    # 'schedule_interval': Frecuencia de ejecución (cada 1 día en este caso)
    schedule_interval=timedelta(days=1),
    
    # 'catchup': Si False, no ejecuta las ejecuciones pasadas al activar el DAG
    catchup=False,
    
    # 'tags': Etiquetas para organizar y filtrar DAGs en la UI
    tags=['ejemplo', 'tutorial']
)

# Verificar que el DAG se creó correctamente
print(f"DAG creado: {dag.dag_id}")
print(f"Schedule: {dag.schedule_interval}")

## 6. Funciones Python para las Tareas

Definimos funciones que serán ejecutadas por nuestras tareas.

In [ ]:
# ============================================
# DEFINIR FUNCIONES PARA LAS TAREAS
# ============================================
# Estas funciones serán ejecutadas por los PythonOperators

# Función 1: Extraer datos (fase Extract del ETL)
def extraer_datos(**context):
    """Simula la extracción de datos de una fuente"""
    print("Extrayendo datos...")
    
    # Simulamos datos extraídos de una base de datos o API
    datos = {'usuarios': 150, 'ventas': 3500, 'productos': 45}
    print(f"Datos extraídos: {datos}")
    
    # El valor retornado se guarda automáticamente en XCom
    return datos


# Función 2: Transformar datos (fase Transform del ETL)
def transformar_datos(**context):
    """Simula la transformación de datos"""
    print("Transformando datos...")
    
    # Aquí aplicaríamos las transformaciones necesarias
    datos_transformados = {
        'total_usuarios': 150,
        'promedio_ventas': 23.33,
        'categoria': 'productos_electronicos'
    }
    print(f"Datos transformados: {datos_transformados}")
    
    return datos_transformados


# Función 3: Cargar datos (fase Load del ETL)
def cargar_datos(**context):
    """Simula la carga de datos en un destino"""
    print("Cargando datos en el destino...")
    print("Datos cargados exitosamente en la base de datos")
    
    return "Carga completada"


# Función 4: Enviar notificación
def enviar_notificacion(**context):
    """Envía una notificación al finalizar el proceso"""
    print("Enviando notificación...")
    print("Notificación enviada: Pipeline ETL completado exitosamente")


print("Funciones definidas correctamente")

## 7. Crear Tareas con Operadores

### PythonOperator
Ejecuta funciones Python

In [ ]:
# ============================================
# CREAR TAREAS CON PYTHONOPERATOR
# ============================================

# Tarea 1: Tarea de inicio (dummy - no hace nada, solo marca el inicio)
inicio = DummyOperator(
    task_id='inicio',  # ID único de la tarea
    dag=dag            # DAG al que pertenece esta tarea
)

# Tarea 2: Extracción de datos
tarea_extraer = PythonOperator(
    task_id='extraer_datos',           # ID único de la tarea
    python_callable=extraer_datos,     # Función Python que ejecutará
    provide_context=True,              # Proporciona el contexto de Airflow a la función
    dag=dag                            # DAG al que pertenece
)

# Tarea 3: Transformación de datos
tarea_transformar = PythonOperator(
    task_id='transformar_datos',       # ID único de la tarea
    python_callable=transformar_datos, # Función a ejecutar
    provide_context=True,              # Permite acceder a XCom y otros datos del contexto
    dag=dag
)

# Tarea 4: Carga de datos
tarea_cargar = PythonOperator(
    task_id='cargar_datos',
    python_callable=cargar_datos,
    provide_context=True,
    dag=dag
)

print("Tareas Python creadas")

### BashOperator
Ejecuta comandos Bash

In [ ]:
# ============================================
# CREAR TAREAS CON BASHOPERATOR
# ============================================

# Tarea con BashOperator: Ejecuta un comando de shell
tarea_bash = BashOperator(
    task_id='verificar_sistema',                        # ID de la tarea
    bash_command='echo "Sistema verificado en $(date)"', # Comando bash a ejecutar
    dag=dag
)

# Tarea de notificación usando PythonOperator
tarea_notificar = PythonOperator(
    task_id='enviar_notificacion',
    python_callable=enviar_notificacion,
    provide_context=True,
    dag=dag
)

# Tarea final (dummy - marca el fin del DAG)
fin = DummyOperator(
    task_id='fin',
    dag=dag
)

print("Todas las tareas creadas")

## 8. Definir Dependencias entre Tareas

Las dependencias determinan el orden de ejecución de las tareas.

### Métodos para definir dependencias:
- `tarea1 >> tarea2` (tarea1 antes de tarea2)
- `tarea1 << tarea2` (tarea2 antes de tarea1)
- `set_upstream()` y `set_downstream()`

In [ ]:
# ============================================
# DEFINIR DEPENDENCIAS ENTRE TAREAS
# ============================================
# El operador '>>' significa "ejecutar antes que"
# Esto crea el flujo de ejecución del DAG

# Flujo completo del pipeline:
# 1. inicio -> 2. verificar_sistema -> 3. extraer_datos -> 
# 4. transformar_datos -> 5. cargar_datos -> 6. enviar_notificacion -> 7. fin

inicio >> tarea_bash >> tarea_extraer >> tarea_transformar >> tarea_cargar >> tarea_notificar >> fin

print("Dependencias configuradas:")
print("inicio -> verificar_sistema -> extraer_datos -> transformar_datos -> cargar_datos -> enviar_notificacion -> fin")

## 9. Ejemplo de DAG con Paralelización

Vamos a crear un DAG donde algunas tareas se ejecutan en paralelo.

In [ ]:
# ============================================
# DAG CON TAREAS PARALELAS
# ============================================
# Este ejemplo muestra cómo ejecutar varias tareas al mismo tiempo

# Crear un nuevo DAG para demostrar paralelización
dag_paralelo = DAG(
    'dag_con_tareas_paralelas',
    default_args=default_args,
    description='DAG con ejecución paralela',
    schedule_interval='@daily',  # Se ejecuta diariamente
    catchup=False
)


# Función 1: Procesar datos de la fuente A
def procesar_fuente_a(**context):
    """Procesa datos de la primera fuente"""
    print("Procesando datos de la fuente A")
    return "Fuente A procesada"


# Función 2: Procesar datos de la fuente B
def procesar_fuente_b(**context):
    """Procesa datos de la segunda fuente"""
    print("Procesando datos de la fuente B")
    return "Fuente B procesada"


# Función 3: Procesar datos de la fuente C
def procesar_fuente_c(**context):
    """Procesa datos de la tercera fuente"""
    print("Procesando datos de la fuente C")
    return "Fuente C procesada"


# Función 4: Consolidar todos los resultados
def consolidar_datos(**context):
    """Consolida los resultados de todas las fuentes"""
    print("Consolidando todas las fuentes")
    return "Consolidación completada"


print("DAG paralelo creado")

In [ ]:
# ============================================
# CREAR Y CONFIGURAR TAREAS PARALELAS
# ============================================

# Tarea inicial
inicio_paralelo = DummyOperator(task_id='inicio', dag=dag_paralelo)

# Crear tres tareas que se ejecutarán EN PARALELO
tarea_fuente_a = PythonOperator(
    task_id='procesar_fuente_a',
    python_callable=procesar_fuente_a,
    dag=dag_paralelo
)

tarea_fuente_b = PythonOperator(
    task_id='procesar_fuente_b',
    python_callable=procesar_fuente_b,
    dag=dag_paralelo
)

tarea_fuente_c = PythonOperator(
    task_id='procesar_fuente_c',
    python_callable=procesar_fuente_c,
    dag=dag_paralelo
)

# Tarea que consolida los resultados (espera a que terminen las 3 anteriores)
tarea_consolidar = PythonOperator(
    task_id='consolidar_datos',
    python_callable=consolidar_datos,
    dag=dag_paralelo
)

# Tarea final
fin_paralelo = DummyOperator(task_id='fin', dag=dag_paralelo)

# ============================================
# DEFINIR DEPENDENCIAS PARALELAS
# ============================================
# La sintaxis [tarea1, tarea2, tarea3] indica que todas se ejecutan en paralelo
# inicio -> [fuente_a, fuente_b, fuente_c] -> consolidar -> fin

inicio_paralelo >> [tarea_fuente_a, tarea_fuente_b, tarea_fuente_c] >> tarea_consolidar >> fin_paralelo

print("Tareas paralelas configuradas")
print("Las tareas A, B y C se ejecutarán en paralelo")

## 10. XCom - Compartir Datos entre Tareas

XCom (cross-communication) permite que las tareas compartan pequeñas cantidades de datos.

In [ ]:
# ============================================
# XCOM - COMPARTIR DATOS ENTRE TAREAS
# ============================================
# XCom (Cross-Communication) permite pasar datos entre tareas

# Crear DAG para demostrar XCom
dag_xcom = DAG(
    'ejemplo_xcom',
    default_args=default_args,
    description='Ejemplo de XCom',
    schedule_interval=None,  # Ejecución manual solamente
    catchup=False
)


def generar_numero(**context):
    """Genera un número aleatorio y lo guarda en XCom"""
    import random
    
    # Generamos un número aleatorio
    numero = random.randint(1, 100)
    print(f"Número generado: {numero}")
    
    # Al hacer 'return', el valor se guarda AUTOMÁTICAMENTE en XCom
    # Otros tasks podrán recuperarlo usando xcom_pull()
    return numero


def procesar_numero(**context):
    """Recupera el número de XCom y lo procesa"""
    
    # 'ti' (Task Instance) contiene métodos para interactuar con XCom
    ti = context['ti']
    
    # xcom_pull() recupera el valor guardado por la tarea 'generar_numero'
    numero = ti.xcom_pull(task_ids='generar_numero')
    
    # Procesamos el número (en este caso, lo multiplicamos por 2)
    resultado = numero * 2
    
    print(f"Número recibido: {numero}")
    print(f"Resultado del procesamiento: {resultado}")
    
    return resultado


# Crear las tareas
tarea_generar = PythonOperator(
    task_id='generar_numero',
    python_callable=generar_numero,
    dag=dag_xcom
)

tarea_procesar = PythonOperator(
    task_id='procesar_numero',
    python_callable=procesar_numero,
    dag=dag_xcom
)

# Definir dependencia: primero generar, luego procesar
tarea_generar >> tarea_procesar

print("DAG con XCom configurado")

## 11. Programación de DAGs (Schedule Interval)

### Opciones de programación:
- `None`: Ejecución manual
- `@once`: Una sola vez
- `@hourly`: Cada hora
- `@daily`: Diario
- `@weekly`: Semanal
- `@monthly`: Mensual
- Expresión cron: `'0 0 * * *'`
- `timedelta`: `timedelta(hours=2)`

In [ ]:
# ============================================
# SCHEDULE INTERVAL - PROGRAMACIÓN DE DAGS
# ============================================
# Diferentes formas de programar cuándo se ejecuta un DAG

from datetime import timedelta

# Diccionario con ejemplos de diferentes programaciones
ejemplos_schedule = {
    # None: El DAG solo se ejecuta manualmente
    'Manual': None,
    
    # @once: Se ejecuta una sola vez
    'Una vez': '@once',
    
    # @hourly: Se ejecuta cada hora (equivale a '0 * * * *')
    'Cada hora': '@hourly',
    
    # @daily: Se ejecuta diariamente a medianoche (equivale a '0 0 * * *')
    'Diario a medianoche': '@daily',
    
    # @weekly: Se ejecuta semanalmente (domingos a medianoche)
    'Semanal': '@weekly',
    
    # timedelta: Ejecutar cada X tiempo desde la última ejecución
    'Cada 2 horas': timedelta(hours=2),
    
    # Expresión CRON: '0 9 * * *' = todos los días a las 9 AM
    # Formato: minuto hora día_mes mes día_semana
    'Diario a las 9 AM': '0 9 * * *',
    
    # '0 8 * * 1-5' = Lunes a Viernes (1-5) a las 8 AM
    'Lunes a Viernes a las 8 AM': '0 8 * * 1-5'
}

print("Ejemplos de Schedule Interval:")
for nombre, schedule in ejemplos_schedule.items():
    print(f"  {nombre}: {schedule}")

## 12. Caso Práctico: Pipeline ETL Completo

Vamos a crear un pipeline ETL real que simula:
1. Extracción de datos de una API
2. Validación de datos
3. Transformación
4. Carga en base de datos
5. Generación de reporte

In [ ]:
# ============================================
# PIPELINE ETL COMPLETO - FUNCIONES
# ============================================

import json
from datetime import datetime

# Crear DAG para un pipeline ETL completo
dag_etl = DAG(
    'pipeline_etl_completo',
    default_args=default_args,
    description='Pipeline ETL completo con validaciones',
    schedule_interval='0 2 * * *',  # Ejecutar diariamente a las 2 AM (expresión CRON)
    catchup=False,
    tags=['etl', 'produccion']
)


def extraer_de_api(**context):
    """Extrae datos de una API (simulado)"""
    print("Conectando a la API...")
    
    # Simulación de respuesta de una API con datos de productos
    datos = [
        {'id': 1, 'nombre': 'Producto A', 'precio': 100, 'stock': 50},
        {'id': 2, 'nombre': 'Producto B', 'precio': 200, 'stock': 30},
        {'id': 3, 'nombre': 'Producto C', 'precio': 150, 'stock': 0},
        {'id': 4, 'nombre': 'Producto D', 'precio': 300, 'stock': 20}
    ]
    print(f"Extraídos {len(datos)} registros")
    
    # Retornar datos para que estén disponibles en XCom
    return datos


def validar_datos(**context):
    """Valida que los datos cumplan con las reglas de negocio"""
    # Obtener datos de la tarea anterior usando XCom
    ti = context['ti']
    datos = ti.xcom_pull(task_ids='extraer_de_api')
    
    datos_validos = []
    datos_invalidos = []
    
    # Validar cada registro según reglas de negocio
    for item in datos:
        # Regla: precio debe ser positivo y stock no negativo
        if item['precio'] > 0 and item['stock'] >= 0:
            datos_validos.append(item)
        else:
            datos_invalidos.append(item)
    
    print(f"Datos válidos: {len(datos_validos)}")
    print(f"Datos inválidos: {len(datos_invalidos)}")
    
    # Retornar solo los datos válidos
    return datos_validos


def transformar_etl(**context):
    """Transforma los datos según los requisitos del negocio"""
    # Recuperar datos validados de la tarea anterior
    ti = context['ti']
    datos = ti.xcom_pull(task_ids='validar_datos')
    
    datos_transformados = []
    
    # Aplicar transformaciones a cada registro
    for item in datos:
        transformado = {
            # Mantener ID
            'producto_id': item['id'],
            
            # Convertir nombre a mayúsculas
            'nombre_producto': item['nombre'].upper(),
            
            # Calcular precio con IVA (21%)
            'precio_con_iva': round(item['precio'] * 1.21, 2),
            
            # Agregar campo booleano de disponibilidad
            'disponible': item['stock'] > 0,
            
            # Categorizar según stock
            'categoria': 'En Stock' if item['stock'] > 0 else 'Agotado',
            
            # Agregar timestamp del procesamiento
            'fecha_proceso': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        datos_transformados.append(transformado)
    
    print(f"Transformados {len(datos_transformados)} registros")
    return datos_transformados


def cargar_en_bd(**context):
    """Simula la carga de datos en una base de datos"""
    # Recuperar datos transformados
    ti = context['ti']
    datos = ti.xcom_pull(task_ids='transformar_etl')
    
    print("Conectando a la base de datos...")
    print(f"Insertando {len(datos)} registros...")
    
    # Simular inserciones en la base de datos
    for item in datos:
        print(f"  INSERT: {item['nombre_producto']} - ${item['precio_con_iva']}")
    
    print("Carga completada exitosamente")
    
    # Retornar cantidad de registros cargados
    return len(datos)


def generar_reporte(**context):
    """Genera un reporte resumen del proceso ETL"""
    # Recuperar información de la tarea de carga
    ti = context['ti']
    registros_cargados = ti.xcom_pull(task_ids='cargar_en_bd')
    
    # Crear reporte formateado
    reporte = f"""
    ========================================
    REPORTE ETL - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    ========================================
    Registros procesados: {registros_cargados}
    Estado: COMPLETADO
    Pipeline: pipeline_etl_completo
    ========================================
    """
    print(reporte)
    
    return reporte


print("Funciones ETL definidas")

In [ ]:
# ============================================
# PIPELINE ETL COMPLETO - CREAR TAREAS
# ============================================

# Tarea 1: Inicio del pipeline
inicio_etl = DummyOperator(task_id='inicio', dag=dag_etl)

# Tarea 2: Extracción de datos desde API
extraer_api = PythonOperator(
    task_id='extraer_de_api',
    python_callable=extraer_de_api,
    dag=dag_etl
)

# Tarea 3: Validación de datos
validar = PythonOperator(
    task_id='validar_datos',
    python_callable=validar_datos,
    dag=dag_etl
)

# Tarea 4: Transformación de datos
transformar = PythonOperator(
    task_id='transformar_etl',
    python_callable=transformar_etl,
    dag=dag_etl
)

# Tarea 5: Carga en base de datos
cargar = PythonOperator(
    task_id='cargar_en_bd',
    python_callable=cargar_en_bd,
    dag=dag_etl
)

# Tarea 6: Generar reporte del proceso
reporte = PythonOperator(
    task_id='generar_reporte',
    python_callable=generar_reporte,
    dag=dag_etl
)

# Tarea 7: Fin del pipeline
fin_etl = DummyOperator(task_id='fin', dag=dag_etl)

# ============================================
# DEFINIR FLUJO DEL PIPELINE ETL
# ============================================
# Flujo secuencial completo del ETL:
# inicio -> extraer -> validar -> transformar -> cargar -> reporte -> fin

inicio_etl >> extraer_api >> validar >> transformar >> cargar >> reporte >> fin_etl

print("Pipeline ETL completo configurado")

## 13. Visualización de la Estructura del DAG

In [ ]:
# ============================================
# VISUALIZAR ESTRUCTURA DE LOS DAGS CREADOS
# ============================================

# Lista con todos los DAGs que hemos creado
dags_creados = [
    ('mi_primer_dag', dag),
    ('dag_con_tareas_paralelas', dag_paralelo),
    ('ejemplo_xcom', dag_xcom),
    ('pipeline_etl_completo', dag_etl)
]

print("\n" + "="*60)
print("RESUMEN DE DAGs CREADOS")
print("="*60)

# Iterar sobre cada DAG y mostrar su información
for nombre, dag_obj in dags_creados:
    print(f"\nDAG: {nombre}")
    print(f"  Descripción: {dag_obj.description}")
    print(f"  Schedule: {dag_obj.schedule_interval}")
    print(f"  Número de tareas: {len(dag_obj.tasks)}")
    # Listar los IDs de todas las tareas del DAG
    print(f"  Tareas: {[t.task_id for t in dag_obj.tasks]}")

## 14. Comandos Útiles de Airflow

### Comandos CLI básicos:

```bash
# Inicializar la base de datos
airflow db init

# Crear usuario admin
airflow users create \
    --username admin \
    --firstname Admin \
    --lastname User \
    --role Admin \
    --email admin@example.com

# Iniciar el webserver
airflow webserver --port 8080

# Iniciar el scheduler
airflow scheduler

# Listar DAGs
airflow dags list

# Probar una tarea específica
airflow tasks test <dag_id> <task_id> <execution_date>

# Ejecutar un DAG manualmente
airflow dags trigger <dag_id>
```

## 15. Mejores Prácticas

### 1. Diseño de DAGs
- Mantén los DAGs simples y enfocados
- Evita dependencias circulares
- Usa nombres descriptivos para tareas

### 2. Manejo de Errores
- Configura reintentos apropiados
- Implementa manejo de excepciones
- Usa alertas para fallos críticos

### 3. Rendimiento
- No uses XCom para datos grandes
- Paraleliza tareas cuando sea posible
- Configura pools para limitar concurrencia

### 4. Monitoreo
- Implementa logs detallados
- Usa SLAs para tareas críticas
- Monitorea el estado de los DAGs regularmente

## 19. Ejercicios Prácticos

### Ejercicio 1: DAG Simple
Crea un DAG que:
1. Lea un archivo CSV
2. Filtre datos por una condición
3. Guarde el resultado

### Ejercicio 2: DAG con Branching
Crea un DAG que ejecute diferentes ramas según una condición

### Ejercicio 3: Pipeline de ML
Crea un DAG que:
1. Cargue datos
2. Entrene un modelo
3. Evalúe el modelo
4. Guarde el mejor modelo

### Ejercicio 4: TaskFlow API Challenge
Convierte un DAG tradicional a TaskFlow API

### Ejercicio 5: Sensor Personalizado
Crea un sensor que espere a que una API específica esté disponible

### Ejercicio 6: Pipeline con Hooks
Crea un pipeline que:
1. Use HttpHook para obtener datos de una API
2. Procese los datos
3. Use un Hook de base de datos para guardar los resultados

In [ ]:
from airflow.decorators import dag, task
from datetime import datetime
import json

@dag(
    dag_id='pipeline_completo_avanzado',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
    description='Pipeline que combina TaskFlow, Sensors y Hooks',
    tags=['completo', 'avanzado', 'produccion']
)
def advanced_pipeline():
    """
    Pipeline avanzado que demuestra:
    - TaskFlow API para código limpio
    - Sensors para esperar condiciones
    - Hooks para integraciones externas
    """
    
    @task
    def setup_pipeline():
        """Inicializa el pipeline"""
        print("="*60)
        print("INICIANDO PIPELINE AVANZADO")
        print(f"Timestamp: {datetime.now()}")
        print("="*60)
        return {'status': 'initialized', 'timestamp': str(datetime.now())}
    
    @task
    def fetch_from_api():
        """Simula fetch de API con Hook"""
        print("\n[1] Obteniendo datos de API...")
        # En producción: HttpHook(http_conn_id='api_prod').run()
        data = {
            'users': [
                {'id': 1, 'name': 'Alice', 'purchases': 150},
                {'id': 2, 'name': 'Bob', 'purchases': 200},
                {'id': 3, 'name': 'Charlie', 'purchases': 175}
            ],
            'timestamp': str(datetime.now())
        }
        print(f"   Obtenidos {len(data['users'])} usuarios")
        return data
    
    @task
    def validate_and_transform(data: dict):
        """Valida y transforma los datos"""
        print("\n[2] Validando y transformando datos...")
        users = data['users']
        
        # Validación
        valid_users = [u for u in users if u['purchases'] > 0]
        print(f"   Usuarios válidos: {len(valid_users)}/{len(users)}")
        
        # Transformación
        transformed = [
            {
                'user_id': u['id'],
                'user_name': u['name'].upper(),
                'total_purchases': u['purchases'],
                'category': 'premium' if u['purchases'] > 180 else 'standard',
                'processed_at': str(datetime.now())
            }
            for u in valid_users
        ]
        
        print(f"   Transformados {len(transformed)} registros")
        return transformed
    
    @task
    def calculate_metrics(users: list):
        """Calcula métricas del negocio"""
        print("\n[3] Calculando métricas...")
        
        total_purchases = sum(u['total_purchases'] for u in users)
        premium_users = len([u for u in users if u['category'] == 'premium'])
        
        metrics = {
            'total_users': len(users),
            'total_revenue': total_purchases,
            'average_purchase': total_purchases / len(users) if users else 0,
            'premium_users': premium_users,
            'premium_percentage': (premium_users / len(users) * 100) if users else 0
        }
        
        print("   Métricas calculadas:")
        for key, value in metrics.items():
            print(f"     {key}: {value}")
        
        return metrics
    
    @task
    def save_to_database(users: list, metrics: dict):
        """Guarda datos en base de datos"""
        print("\n[4] Guardando en base de datos...")
        # En producción: PostgresHook(postgres_conn_id='db_prod').run()
        
        print(f"   Insertando {len(users)} usuarios...")
        print(f"   Insertando métricas: {metrics['total_revenue']} revenue")
        print("   ✓ Guardado exitoso")
        
        return {'status': 'saved', 'records': len(users)}
    
    @task
    def send_report(save_status: dict, metrics: dict):
        """Envía reporte final"""
        print("\n[5] Generando reporte final...")
        print("="*60)
        print("REPORTE EJECUTIVO - PIPELINE COMPLETADO")
        print("="*60)
        print(f"Estado: {save_status['status'].upper()}")
        print(f"Registros procesados: {save_status['records']}")
        print(f"Revenue total: ${metrics['total_revenue']}")
        print(f"Usuarios premium: {metrics['premium_users']} ({metrics['premium_percentage']:.1f}%)")
        print(f"Compra promedio: ${metrics['average_purchase']:.2f}")
        print("="*60)
        print("Pipeline finalizado exitosamente\n")
    
    # Definir el flujo completo
    init = setup_pipeline()
    api_data = fetch_from_api()
    transformed_data = validate_and_transform(api_data)
    metrics = calculate_metrics(transformed_data)
    save_status = save_to_database(transformed_data, metrics)
    
    # Establecer dependencias
    init >> api_data
    send_report(save_status, metrics)

# Instanciar el DAG
advanced_dag = advanced_pipeline()

print("\n" + "="*60)
print("DAG AVANZADO CREADO EXITOSAMENTE")
print("="*60)
print("Características:")
print("  ✓ TaskFlow API para código limpio")
print("  ✓ Type hints para mejor desarrollo")
print("  ✓ Paso automático de datos entre tareas")
print("  ✓ Estructura modular y escalable")
print("  ✓ Integración con Hooks y Sensors")
print("="*60 + "\n")

## 20. Recursos Adicionales

### Documentación Oficial
- [Apache Airflow Documentation](https://airflow.apache.org/docs/)
- [Airflow Best Practices](https://airflow.apache.org/docs/apache-airflow/stable/best-practices.html)
- [TaskFlow API Tutorial](https://airflow.apache.org/docs/apache-airflow/stable/tutorial_taskflow_api.html)
- [Pythonic DAGs with TaskFlow](https://airflow.apache.org/docs/apache-airflow/stable/tutorial/taskflow.html)
- [TaskFlow Concepts](https://airflow.apache.org/docs/apache-airflow/stable/core-concepts/taskflow.html)

### Tutoriales Externos
- [Apache Airflow Tutorial: Ultimate Guide 2024 - Innowise](https://innowise.com/blog/apache-airflow-introduction/)
- [Getting Started with Apache Airflow - DataCamp](https://www.datacamp.com/tutorial/getting-started-with-apache-airflow)

### Operadores Útiles
- **PythonOperator**: Ejecutar funciones Python
- **BashOperator**: Ejecutar comandos Bash
- **EmailOperator**: Enviar emails
- **HttpOperator**: Llamadas HTTP/API
- **SqlOperator**: Consultas SQL
- **BranchPythonOperator**: Ramificación condicional

### Sensors Comunes
- **FileSensor**: Detectar archivos
- **TimeSensor**: Esperar hasta una hora
- **HttpSensor**: Verificar endpoints
- **ExternalTaskSensor**: Esperar otras tareas

### Hooks Populares
- **HttpHook**: APIs REST
- **PostgresHook**: PostgreSQL
- **MySqlHook**: MySQL
- **S3Hook**: Amazon S3
- **GCSHook**: Google Cloud Storage
- **SlackHook**: Notificaciones Slack

### Providers
- AWS
- Google Cloud
- Azure
- Kubernetes
- Snowflake
- Databricks
- Y muchos más...

In [ ]:
## Conclusión

En este tutorial has aprendido:

✅ Conceptos fundamentales de Apache Airflow  
✅ Cómo crear y configurar DAGs  
✅ Diferentes tipos de operadores  
✅ Manejo de dependencias entre tareas  
✅ Ejecución paralela de tareas  
✅ Compartir datos con XCom  
✅ Programación de DAGs  
✅ Implementación de pipelines ETL completos  
✅ **TaskFlow API (Airflow 2.0+)** - Código más limpio y Pythonic  
✅ **Sensors** - Esperar condiciones antes de continuar  
✅ **Hooks** - Integración con servicios externos  
✅ Mejores prácticas  

### Lo Nuevo en Airflow 2.0+

**TaskFlow API** revoluciona la forma de escribir DAGs:
- Decoradores simples `@dag` y `@task`
- Paso automático de datos (sin XCom manual)
- Type hints para mejor desarrollo
- Código más limpio y mantenible

**Sensors mejorados** con decoradores:
- `@task.sensor` para crear sensors personalizados
- Mejor manejo de recursos con `mode='reschedule'`
- Integración perfecta con TaskFlow

**Hooks modernos**:
- Mejor abstracción de conexiones externas
- Manejo seguro de credenciales
- Integración con cientos de servicios

### Próximos Pasos
1. Practica creando tus propios DAGs con TaskFlow API
2. Explora diferentes Sensors y Hooks
3. Implementa pipelines de datos reales
4. Aprende sobre Airflow en producción
5. Explora integraciones con Cloud Providers
6. Investiga sobre Airflow en Kubernetes

### Recursos Avanzados
- [Understanding TaskFlow API - Restack](https://www.restack.io/docs/airflow-faq-tutorial-taskflow-01)
- [Apache Airflow Components - Restack](https://www.restack.io/docs/airflow-faq-core-concepts-tasks-07)
- Comunidad Airflow en Slack
- Airflow Summit (conferencia anual)

¡Feliz orquestación de flujos de trabajo! 🚀

---

**Nota**: Este notebook ha sido enriquecido con información de:
- Documentación oficial de Apache Airflow
- Tutoriales de la comunidad
- Mejores prácticas de industria
- Ejemplos prácticos de producción

## 18. Hooks en Airflow

### ¿Qué son los Hooks?

Los **Hooks** son interfaces para interactuar con sistemas externos:
- **Abstraen** la conexión a servicios externos
- **Reutilizan** conexiones y credenciales
- **Simplifican** la interacción con APIs, bases de datos, servicios cloud

### Hooks Comunes
- **HttpHook**: API REST
- **PostgresHook**: PostgreSQL
- **MySqlHook**: MySQL
- **S3Hook**: Amazon S3
- **GCSHook**: Google Cloud Storage
- **SlackHook**: Slack

### Ventajas
- Manejo centralizado de conexiones
- Credenciales seguras en Airflow Connections
- Código más limpio y mantenible

In [ ]:
# ============================================
# SENSOR PERSONALIZADO CON @task.sensor
# ============================================

from airflow.decorators import dag, task
from datetime import datetime
import random


@dag(
    dag_id='custom_sensor_ejemplo',
    start_date=datetime(2024, 1, 1),
    schedule_interval=None,  # Solo ejecución manual
    catchup=False
)
def dag_with_custom_sensor():
    """DAG con sensor personalizado usando decorador"""
    
    # ============================================
    # @task.sensor: Crear sensor personalizado
    # ============================================
    @task.sensor(
        poke_interval=10,    # Verificar cada 10 segundos
        timeout=120,         # Timeout después de 2 minutos
        mode='poke'          # Modo de ejecución
    )
    def wait_for_condition() -> bool:
        """Sensor que espera una condición personalizada
        
        IMPORTANTE: Debe retornar bool
        - True: Condición cumplida, continuar
        - False: Condición no cumplida, seguir esperando
        """
        # Simulamos verificar una condición externa (ej: API, archivo, etc)
        condition_met = random.random() > 0.7  # 30% de probabilidad
        
        if condition_met:
            print("✓ Condición cumplida!")
            return True  # Sensor deja de esperar
        else:
            print("✗ Condición no cumplida, esperando...")
            return False  # Sensor sigue verificando
    
    @task
    def process_when_ready():
        """Esta tarea solo se ejecuta cuando el sensor retorna True"""
        print("Sensor detectó que la condición se cumplió")
        print("Iniciando procesamiento...")
        return "Proceso completado"
    
    # ============================================
    # FLUJO CON SENSOR PERSONALIZADO
    # ============================================
    # El proceso solo inicia cuando wait_for_condition() retorna True
    wait_for_condition() >> process_when_ready()


# Instanciar el DAG
custom_sensor_dag = dag_with_custom_sensor()

print("Sensor personalizado con decorador @task.sensor creado")

### Sensor Personalizado con Decorador @task.sensor

In [ ]:
# ============================================
# SENSORS - ESPERAR CONDICIONES
# ============================================

from airflow.decorators import dag, task
from airflow.sensors.filesystem import FileSensor
from airflow.sensors.time_sensor import TimeSensor
from datetime import datetime
import os


@dag(
    dag_id='ejemplo_sensors',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@hourly',
    catchup=False,
    tags=['sensors', 'avanzado']
)
def pipeline_with_sensors():
    """DAG que demuestra el uso de Sensors"""
    
    # ============================================
    # TIMESENSOR: Espera hasta una hora específica
    # ============================================
    wait_for_morning = TimeSensor(
        task_id='wait_for_9am',
        # target_time: Hora objetivo a esperar
        target_time=datetime.strptime('09:00:00', '%H:%M:%S').time(),
        # poke_interval: Cada cuánto verificar (300 seg = 5 min)
        poke_interval=300,
        # mode='reschedule': Libera el worker mientras espera (más eficiente)
        mode='reschedule'
    )
    
    @task
    def create_temp_file():
        """Crea archivo temporal para demostración"""
        filepath = '/tmp/airflow_sensor_test.txt'
        with open(filepath, 'w') as f:
            f.write('Archivo de prueba para sensor')
        print(f"Archivo creado: {filepath}")
        return filepath
    
    # ============================================
    # FILESENSOR: Espera a que exista un archivo
    # ============================================
    wait_for_file = FileSensor(
        task_id='wait_for_data_file',
        # filepath: Ruta del archivo a esperar
        filepath='/tmp/airflow_sensor_test.txt',
        # poke_interval: Verificar cada 30 segundos
        poke_interval=30,
        # timeout: Tiempo máximo de espera (600 seg = 10 min)
        timeout=600,
        # mode='poke': Ocupa el worker mientras espera
        mode='poke'
    )
    
    @task
    def process_file():
        """Procesa el archivo una vez detectado por el sensor"""
        print("Archivo detectado, procesando...")
        with open('/tmp/airflow_sensor_test.txt', 'r') as f:
            content = f.read()
        print(f"Contenido: {content}")
        return "Procesamiento completado"
    
    @task
    def cleanup():
        """Limpia archivos temporales"""
        filepath = '/tmp/airflow_sensor_test.txt'
        if os.path.exists(filepath):
            os.remove(filepath)
            print(f"Archivo eliminado: {filepath}")
    
    # ============================================
    # DEFINIR FLUJO CON SENSORS
    # ============================================
    # El sensor 'wait_for_file' bloquea la ejecución hasta que el archivo exista
    file_path = create_temp_file()
    file_path >> wait_for_file >> process_file() >> cleanup()


# Instanciar el DAG
sensors_dag = pipeline_with_sensors()

print("DAG con Sensors creado")
print("Los sensors esperan condiciones antes de continuar")

## 17. Sensors en Airflow

### ¿Qué son los Sensors?

Los **Sensors** son un tipo especial de operador que:
- **Esperan** a que se cumpla una condición específica
- **Verifican periódicamente** (polling) el estado
- **Bloquean** la ejecución hasta que la condición se cumple

### Tipos Comunes de Sensors
- **FileSensor**: Espera a que exista un archivo
- **ExternalTaskSensor**: Espera a que otra tarea termine
- **TimeSensor**: Espera hasta una hora específica
- **HttpSensor**: Espera respuesta HTTP
- **SqlSensor**: Espera resultado SQL

### Parámetros Importantes
- `poke_interval`: Tiempo entre verificaciones (segundos)
- `timeout`: Tiempo máximo de espera
- `mode`: 'poke' (ocupa worker) o 'reschedule' (libera worker)

In [ ]:
# ============================================
# PIPELINE ETL CON TASKFLOW API
# ============================================

from airflow.decorators import dag, task
from datetime import datetime
from typing import List, Dict  # Para type hints


@dag(
    dag_id='etl_taskflow_completo',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
    description='Pipeline ETL usando TaskFlow API',
    tags=['taskflow', 'etl', 'avanzado']
)
def etl_pipeline_taskflow():
    """Pipeline ETL completo usando decoradores"""
    
    @task
    def extract_from_sources() -> List[Dict]:
        """Extrae datos - Retorna lista de diccionarios"""
        print("Extrayendo datos...")
        return [
            {'id': 1, 'value': 100, 'category': 'A'},
            {'id': 2, 'value': 200, 'category': 'B'},
            {'id': 3, 'value': 150, 'category': 'A'},
            {'id': 4, 'value': 300, 'category': 'C'}
        ]
    
    @task
    def validate_data(data: List[Dict]) -> List[Dict]:
        """Valida datos - Recibe y retorna lista de diccionarios"""
        print(f"Validando {len(data)} registros...")
        # List comprehension para filtrar datos válidos
        valid_data = [item for item in data if item['value'] > 0]
        print(f"Registros válidos: {len(valid_data)}")
        return valid_data
    
    @task
    def transform_by_category(data: List[Dict]) -> Dict[str, List[Dict]]:
        """Agrupa y transforma por categoría - Retorna diccionario"""
        print("Transformando datos por categoría...")
        categories = {}
        
        # Agrupar por categoría y duplicar valores
        for item in data:
            cat = item['category']
            if cat not in categories:
                categories[cat] = []
            categories[cat].append({
                'id': item['id'],
                'value_doubled': item['value'] * 2,  # Transformación
                'category': cat
            })
        return categories
    
    @task
    def calculate_statistics(grouped_data: Dict[str, List[Dict]]) -> Dict:
        """Calcula estadísticas por categoría"""
        print("Calculando estadísticas...")
        stats = {}
        
        # Calcular totales y promedios por categoría
        for category, items in grouped_data.items():
            total = sum(item['value_doubled'] for item in items)
            count = len(items)
            stats[category] = {
                'total': total,
                'count': count,
                'average': total / count if count > 0 else 0
            }
        return stats
    
    @task
    def load_to_database(stats: Dict) -> str:
        """Carga estadísticas a base de datos"""
        print("Cargando datos a la base de datos...")
        for category, data in stats.items():
            print(f"  Categoría {category}: Total={data['total']}, Promedio={data['average']:.2f}")
        return "Carga completada exitosamente"
    
    @task
    def send_notification(status: str) -> None:
        """Envía notificación final"""
        print(f"\n{'='*50}")
        print(f"NOTIFICACIÓN: {status}")
        print(f"Pipeline completado en: {datetime.now()}")
        print(f"{'='*50}\n")
    
    # ============================================
    # DEFINIR EL FLUJO DEL PIPELINE
    # ============================================
    # Cada variable contiene el resultado de la tarea anterior
    # El paso de datos es AUTOMÁTICO
    
    raw_data = extract_from_sources()           # 1. Extraer
    valid_data = validate_data(raw_data)        # 2. Validar (recibe raw_data)
    grouped_data = transform_by_category(valid_data)  # 3. Transformar (recibe valid_data)
    statistics = calculate_statistics(grouped_data)   # 4. Calcular stats (recibe grouped_data)
    status = load_to_database(statistics)       # 5. Cargar (recibe statistics)
    send_notification(status)                   # 6. Notificar (recibe status)


# Instanciar el DAG
etl_taskflow_dag = etl_pipeline_taskflow()

print("Pipeline ETL con TaskFlow API creado")
print("Ventajas: Type hints, paso automático de datos, código más legible")

### Pipeline ETL Completo con TaskFlow API

In [ ]:
# ============================================
# TASKFLOW API (Airflow 2.0+) - MODERNO
# ============================================
# Forma moderna y más limpia de crear DAGs

from airflow.decorators import dag, task
from datetime import datetime


# @dag: Decorador que convierte una función en un DAG
@dag(
    dag_id='taskflow_ejemplo',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
    tags=['taskflow', 'moderno']
)
def pipeline_taskflow():
    """Función que define el DAG completo"""
    
    # @task: Decorador que convierte una función en una tarea
    @task
    def extract() -> dict:
        """Extrae datos - El tipo de retorno es explícito"""
        return {'data': 100}
    
    @task
    def transform(data: dict) -> int:
        """Transforma datos - Recibe data como parámetro directamente
        ¡No necesitamos XCom manual!"""
        return data['data'] * 2
    
    @task
    def load(value: int) -> None:
        """Carga datos - Type hints hacen el código más claro"""
        print(f"Valor final: {value}")
    
    # ============================================
    # DEFINIR EL FLUJO - Syntax Pythonic
    # ============================================
    # El paso de datos es AUTOMÁTICO entre tareas
    data = extract()                # Llama a extract()
    transformed = transform(data)   # Pasa 'data' automáticamente
    load(transformed)               # Pasa 'transformed' automáticamente


# Instanciar el DAG (necesario para que Airflow lo detecte)
dag_taskflow = pipeline_taskflow()

print("TaskFlow API: Código más limpio y Pythonic")
print("El paso de datos entre tareas es automático")

In [ ]:
# ============================================
# FORMA TRADICIONAL (Airflow 1.x)
# ============================================
# Este es el método antiguo de crear DAGs en Airflow

from airflow import DAG
from airflow.operators.python import PythonOperator


def extract_tradicional(**context):
    """Función que extrae datos"""
    return {'data': 100}


def transform_tradicional(**context):
    """Función que transforma datos"""
    # PROBLEMA: Necesitamos usar XCom manualmente
    ti = context['ti']  # Obtener Task Instance
    data = ti.xcom_pull(task_ids='extract')  # Recuperar datos manualmente
    return data['data'] * 2


# Crear DAG tradicional usando 'with' statement
with DAG('tradicional', start_date=datetime(2024, 1, 1)) as dag_trad:
    
    # Crear tarea de extracción
    extract = PythonOperator(
        task_id='extract',
        python_callable=extract_tradicional
    )
    
    # Crear tarea de transformación
    transform = PythonOperator(
        task_id='transform',
        python_callable=transform_tradicional
    )
    
    # Definir dependencias
    extract >> transform

print("Forma tradicional: Requiere más código y manejo manual de XCom")

In [ ]:
# Espacio para tus ejercicios

# Ejercicio 1: Tu código aquí


# Ejercicio 2: Tu código aquí


# Ejercicio 3: Tu código aquí


## 17. Recursos Adicionales

### Documentación Oficial
- [Apache Airflow Documentation](https://airflow.apache.org/docs/)
- [Airflow Best Practices](https://airflow.apache.org/docs/apache-airflow/stable/best-practices.html)

### Operadores Útiles
- **PythonOperator**: Ejecutar funciones Python
- **BashOperator**: Ejecutar comandos Bash
- **EmailOperator**: Enviar emails
- **HttpOperator**: Llamadas HTTP/API
- **SqlOperator**: Consultas SQL
- **BranchPythonOperator**: Ramificación condicional

### Providers
- AWS
- Google Cloud
- Azure
- Kubernetes
- Snowflake
- Y muchos más...

## Conclusión

En este tutorial has aprendido:

✅ Conceptos fundamentales de Apache Airflow  
✅ Cómo crear y configurar DAGs  
✅ Diferentes tipos de operadores  
✅ Manejo de dependencias entre tareas  
✅ Ejecución paralela de tareas  
✅ Compartir datos con XCom  
✅ Programación de DAGs  
✅ Implementación de pipelines ETL completos  
✅ Mejores prácticas  

### Próximos Pasos
1. Practica creando tus propios DAGs
2. Explora diferentes operadores
3. Implementa pipelines de datos reales
4. Aprende sobre Airflow en producción
5. Explora integraciones con otras herramientas

¡Feliz orquestación de flujos de trabajo! 🚀